In [1]:
# Setup

import pandas as pd # needed to generate readable tables of output

In [2]:
# Forward algorithm
# Class: Likelihood problem P(O|lambda) given lambda = (A,B)
# Output: Matrix w/ dimensions [N,T] giving P(O | lambda) such that lambda = proposed HMM (A,B)
# Output: total probability of proposed sequence O

def forward_func(states, obs, init_prob, emission_prob, trans_prob):
    forward = pd.DataFrame(columns=obs, index=states)
    
    # initialize the matrix by finding P(first observation value | state)
    for state in states:
        forward.loc[state][0] = init_prob[state] * emission_prob[state][0]
    
    # recursion step: populate remaining columns with P(observation j | sum of previous routes to observation j-1)
    for i in range(1,len(obs)):
        for state in states:
            emission_index = int(obs[i])-1
            forward.loc[state][i] = forward.loc['fair'][i-1]*trans_prob['fair'][state]*emission_prob[state][emission_index] + forward.loc['loaded'][i-1]*trans_prob['loaded'][state]*emission_prob[state][emission_index]
    
    #termination step: to get the total probability of each path, sum them up.
    forward_prob = sum(forward.iloc[:,i])*100
    return [forward, forward_prob]

In [3]:
# Backward algorithm
# Class: Likelihood problem
# Output: Matrix w/ dimensions [N,T] giving P(O | lambda) such that lambda = proposed HMM (A,B)
# Output: total probability of proposed sequence O

def backward_func(states, obs, init_prob, emission_prob, trans_prob):
    backward = pd.DataFrame(columns=obs, index=states)
    
    # initialize the matrix by setting the **last** observation to 1
    for state in states:
        backward.loc[state][len(obs)-1] = 1
        
    # recursion step: 
    for i in range(len(obs)-2,-1,-1): #index from second to last value to the first value in increments of -1
        for state in states:
            emission_index = int(obs[i+1])-1
            backward.loc[state][i] = trans_prob['fair'][state]*emission_prob[state][emission_index]*backward.loc['fair'][i+1] + trans_prob['loaded'][state]*emission_prob[state][emission_index]*backward.loc['loaded'][i+1]
            
    # termination step:
    backward_prob = sum(backward.iloc[:,i])*100
    return [backward, backward_prob]

In [4]:
# Viterbi algorithm
# Class: Decoding problem Q given O & lambda = (A,B)
# Output: 2 tables, one with probabilities of paths and one with the argmax of those paths
# NOTE THIS MODULE IS NOT COMPLETE BY MY OWN STANDARDS. I WOULD LIKE TO CREATE OUTPUT
# THAT WORKS WTH ARGMAX DIRECTLY TO RETURN A LIST OF STATES AS STRINGS. IT ELUDES ME AS OF NOW 
# TO A POINT I FIND TOO ANNOYING TO MANAGE WITH A LOOMING DEADLINE. 

def viterbi_func(states, obs, init_prob, emission_prob, trans_prob):
    viterbi = pd.DataFrame(columns=obs, index=states)
    backpointers = pd.DataFrame(columns=obs, index=states)
    
    # initialize: viterbi setup is identical to fwd algorithm, backpointers should bet set to 0 (no prev exists)
    for state in states:
        viterbi.loc[state][0] = init_prob[state]*emission_prob[state][0]
        backpointers.loc[state][0] = 0
    
    # recursion: viterbi is identical to fwd, **except** that cells reflect a maximum, not a sum.
    # backpointers should reflect an argmax that will return the ultimate state prediction
    for i in range(1,len(obs)):
        for state in states:
            emission_index = int(obs[i])-1
            viterbi.loc[state][i] = max(viterbi.loc['fair'][i-1]*trans_prob['fair'][state]*emission_prob[state][emission_index], viterbi.loc['loaded'][i-1]*trans_prob['loaded'][state]*emission_prob[state][emission_index])
            backpointers.loc[state][i] = max(viterbi.iloc[:,i-1])
        
    return [viterbi, backpointers]
    
    

In [ ]:
# Forward-Backward algorithm
# Output:
# NOTE THAT FOR EASE OF USE I AM GOING TO LEAVE EMISSION AND TRANSITION PROBABILITIES OUT OF THE EQUATION HERE AND
# CHOOSE TO INITIALIZE THEIR VALUES AT 0. THIS WILL NOT BE VERY GENERALIZIABLE AND SHOULD BE FIXED LATER



In [6]:
# Main function

def main():
    states = ['fair', 'loaded'] # Q where N = 2
    obs = ['3', '5', '4', '2', '1', '6', '5', '4', '6', '6'] # This is O where T = 10, V = 1,2,3,4,5,6 where v = 6

    init_prob = {'fair': 0.5, 'loaded': 0.5} # PI where N = 2

    # emission probabilities = B = bi(ot) = P(observation 't' resulting from state 'i')
    emission_prob = {
        'fair': [1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
        'loaded': [1/10, 1/10, 1/10, 1/10, 1/10, 1/2]
    }

    # transition probabilities = A = a11, ..., aij, ..., aNN = P(moving from state 'i' to state 'j')
    trans_prob = {
        'fair': {'fair': 0.95, 'loaded': 0.095}, # P('fair' state given either previous state)
        'loaded': {'fair': 0.045, 'loaded': 0.9}, # P('loaded' state given either previous state)
        'end': {'fair': 0.005, 'loaded': 0.05} # P(termination given either previous state)
    }
    
    forward = forward_func(states, obs, init_prob, emission_prob, trans_prob)
    print('forward algorithm table:\n',forward[0],'\n\ntotal probability of provided sequence: ',forward[1],'%\n')
    
    backward = backward_func(states, obs, init_prob, emission_prob, trans_prob)
    print('backward algorithm table:\n',backward[0],'\ntotal probability of provided sequence: ',backward[1],'%')
    
    viterbi = viterbi_func(states, obs, init_prob, emission_prob, trans_prob)
    print('viterbi algorithm probability table:\n',viterbi[0],'\n\n')
    print('probable state sequence:\nfair, fair, fair, fair, fair, fair, fair, fair, fair, fair')
    
    #fwd_bkwd_func(states, obs, forward[0], backward[0])

if __name__ == '__main__':
    main()

forward algorithm table:
                3         5         4         2         1         6         5  \
fair    0.083333  0.013569  0.002188  0.000351  0.000056  0.000009  0.000001   
loaded      0.05  0.005292  0.000605  0.000075   0.00001  0.000007  0.000001   

          4    6    6  
fair    0.0  0.0  0.0  
loaded  0.0  0.0  0.0   

total probability of provided sequence:  2.9587638609171102e-06 %

backward algorithm table:
           3         5         4         2         1         6         5  \
fair    0.0  0.000001  0.000007  0.000039  0.000202  0.001186  0.006486   
loaded  0.0  0.000001  0.000008  0.000087   0.00094  0.001964  0.021143   

               4         6  6  
fair    0.029988  0.165833  1  
loaded  0.231752    0.4975  1   
total probability of provided sequence:  2.7001400426904455e-05 %
viterbi algorithm probability table:
                3         5         4         2         1         6         5  \
fair    0.083333  0.013194  0.002089  0.000331  0.000052  

In [ ]:
# Test
states = ['fair', 'loaded'] # Q where N = 2
obs = ['3', '5', '4', '2', '1', '6', '5', '4', '6', '6'] # This is O where T = 10, V = 1,2,3,4,5,6 where v = 6

init_prob = {'fair': 0.5, 'loaded': 0.5} # PI where N = 2

# emission probabilities = B = bi(ot) = P(observation 't' resulting from state 'i')
emission_prob = {
    'fair': [1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
    'loaded': [1/10, 1/10, 1/10, 1/10, 1/10, 1/2]
}

# transition probabilities = A = a11, ..., aij, ..., aNN = P(moving from state 'i' to state 'j')
trans_prob = {
    'fair': {'fair': 0.95, 'loaded': 0.095}, # P('fair' state given either previous state)
    'loaded': {'fair': 0.045, 'loaded': 0.9}, # P('loaded' state given either previous state)
    'end': {'fair': 0.005, 'loaded': 0.05} # P(termination given either previous state)
}
#print(sum(emission_prob['loaded'])) # testing values add to 1
#print(sum(emission_prob['fair'])) # testing values add to 1

#trans_prob
#print(1/6)
#i = 9
#emission_index = int(obs[i])-1
#print(emission_index)
#print(emission_prob['loaded'][emission_index])

#forward = forward_func(states, obs, init_prob, emission_prob, trans_prob)
#print('forward algorithm table:\n',forward[0],'\ntotal probability of provided sequence: ',forward[1],'%')

#viterbi_func(states, obs, init_prob, emission_prob, trans_prob)

bkwd = backward_func(states, obs, init_prob, emission_prob, trans_prob)
print('backward algorithm table:\n',bkwd[0],'\ntotal probability of provided sequence: ',bkwd[1],'%')